# Slowly Changing Dimensions in Data Engineering

#### Implementation Steps
#### 1. Create a Spark Session
#### 2. Create the Initial DataFrame

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.sql.functions import *  # Import the function
spark = SparkSession.builder.getOrCreate()


spark = SparkSession.builder.getOrCreate()

#### 3. Create the New DataFrame


In [ ]:
schema = StructType([
    StructField('CustomerID', IntegerType(), True),
    StructField('CustomerName', StringType(), False),
    StructField('City', StringType(), False),
    StructField('SignUpDate', DateType(), False)
])

# Create data for the initial DataFrame (dates as strings; will parse to DateType)
data = [
    (1, "John Doe", "New York", "2022-01-15"),
    (2, "Jane Smith", "Los Angeles", "2022-02-20"),
    (3, "Sam Brown", "Chicago", "2022-03-25")
]

# create DataFrame without strict StructType to avoid Arrow conversion errors; set column names then cast/parse
createDf = spark.createDataFrame(data, ['CustomerID','CustomerName','City','SignUpDate'])
# Ensure CustomerID is integer and parse SignUpDate to DateType
createDf = createDf.withColumn('CustomerID', col('CustomerID').cast('int'))
createDf = createDf.withColumn('SignUpDate', to_date(col('SignUpDate'), 'yyyy-MM-dd'))
createDf.show()


+----------+------------+-----------+----------+
|CustomerID|CustomerName|       City|SignUpDate|
+----------+------------+-----------+----------+
|         1|    John Doe|   New York|2022-01-15|
|         2|  Jane Smith|Los Angeles|2022-02-20|
|         3|   Sam Brown|    Chicago|2022-03-25|
+----------+------------+-----------+----------+



####   Write initial dimension to delta table

In [15]:
# Use an absolute Windows path (file URI) so Delta can write locally
dim_path = 'file:///C:/Users/srima/Documents/learning/Technology/Data Engineering/PySpark/DalyPySpark/SCDtemp/delta/customer_dim'
# Write as Delta table (requires delta-spark to be installed and Spark configured with Delta)
createDf.write.format("delta").mode("overwrite").save(dim_path)
# Read back to verify
df = spark.read.format("delta").load(dim_path)
df.show()

UnknownException: (org.apache.spark.SparkClassNotFoundException) [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Make sure the provider name is correct and the package is properly registered and compatible with your Spark version. SQLSTATE: 42K02

JVM stacktrace:
org.apache.spark.SparkClassNotFoundException
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:681)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:740)
	at org.apache.spark.sql.classic.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:626)
	at org.apache.spark.sql.classic.DataFrameWriter.saveInternal(DataFrameWriter.scala:135)
	at org.apache.spark.sql.classic.DataFrameWriter.save(DataFrameWriter.scala:118)
	at org.apache.spark.sql.connect.planner.SparkConnectPlanner.handleWriteOperation(SparkConnectPlanner.sca

#### Implement SCD Type 0 Logic


In [ ]:
# Print Spark version to help pick delta-spark compatibility
print('Spark version:', spark.version)

Spark version: 4.0.0
